In [1]:
import pandas
from create_train_test_val_maps import *

%load_ext autoreload
%autoreload 2

In [ ]:
s_to_date = lambda x: pandas.to_datetime(x, format='%m/%d/%Y %H:%M:%S')
snapshots = pandas.read_csv('/home/cs231n/data/selected_snapshots_full_deduped_backfilled_whitened.csv', converters={'Event DateTime':s_to_date})

s_to_date = lambda x: pandas.to_datetime(x, format='%Y/%m/%d %H:%M:%S')
ok_snapshots = pandas.read_csv('/home/cs231n/data/selected_snapshots_ok_deduped_backfilled_whitened.csv', converters={'Event DateTime':s_to_date})

In [2]:
snapshots = pandas.read_pickle('selected_snapshots_full_deduped_backfilled_whitened.pkl')
ok_snapshots = pandas.read_pickle('selected_snapshots_ok_full_deduped_backfilled_whitened.pkl')
#snapshots.to_pickle('selected_snapshots_full_deduped_backfilled_whitened.pkl')
#ok_snapshots.to_pickle('selected_snapshots_ok_full_deduped_backfilled_whitened.pkl')

In [3]:
ata6_train_repairs = pandas.read_csv('ATA6_train.csv')
ata6_val_repairs = pandas.read_csv('ATA6_val.csv')
ata6_test_repairs = pandas.read_csv('ATA6_test.csv')

# removed 'Veh Ref ID' from category_var
category_var = ['Event Type Description','Brake Switch','Clutch Switch','Cruise Status','Dpf Regen Inhibit Sw', 
                'Dpf Thermal Mngmnt','Eng Coolant Level','DTCID']
for cat in category_var:
    snapshots[cat] = snapshots[cat].astype('category').cat.codes
    ok_snapshots[cat] = ok_snapshots[cat].astype('category').cat.codes

try:
    snapshots = snapshots.drop(['DTC'],1)
except:
    print('')
try:
    snapshots = snapshots.drop(['Unnamed: 0'],1)
except:
    print('')
try:
    snapshots = snapshots.drop(['Ignition Cycle Counter'],1)
except:
    print('')
    
try:
    ok_snapshots = ok_snapshots.drop(['DTC'],1)
except:
    print('')
try:
    ok_snapshots = ok_snapshots.drop(['Unnamed: 0'],1)
except:
    print('')
try:
    ok_snapshots = ok_snapshots.drop(['Ignition Cycle Counter'],1)
except:
    print('')

train_veh_ids = ata6_train_repairs['Chassis Reference Number'].unique().astype(int)
val_veh_ids = ata6_val_repairs['Chassis Reference Number'].unique().astype(int)
test_veh_ids = ata6_test_repairs['Chassis Reference Number'].unique().astype(int)

all_veh_ids = ok_snapshots['Veh Ref ID'].unique().astype(int)
train_ok_ids = all_veh_ids[0:110]
val_ok_ids = all_veh_ids[110:130]
test_ok_ids = all_veh_ids[130:144]

ids_tuple = (train_veh_ids, val_veh_ids, test_veh_ids)
ids_ok_tuple = (train_ok_ids, val_ok_ids, test_ok_ids)
repairs_tuple = (ata6_train_repairs, ata6_val_repairs, ata6_test_repairs)

In [13]:
selected_codes = [0,45021, 44004, 43004, 45008, 45002, 45007] # first code is the ok
selected_codes = list(np.sort(selected_codes))
train_map_revived = val_combined
models = {}
codes = []
code_window_map = {}
cat_col = [0, 4, 8, 10, 11, 12, 16, 47]
for vehicleID in train_map_revived.keys():
    for ATA6code in train_map_revived[vehicleID].keys():
        if ATA6code not in selected_codes:
            continue
        if ATA6code not in code_window_map:
            code_window_map[ATA6code] = {}
        for time_window in train_map_revived[vehicleID][ATA6code].keys():
            if time_window not in code_window_map[ATA6code]:
                code_window_map[ATA6code][time_window] = []
            for sequence_of_snapshots in train_map_revived[vehicleID][ATA6code][time_window]:
                code_window_map[ATA6code][time_window].append(len(sequence_of_snapshots))
                #code_window_map[ATA6code][time_window].append(1)

In [14]:
ATA6code = 44004
time_window = 0
sum(code_window_map[ATA6code][time_window])
num_snapshots = []
for ATA6code in code_window_map.keys():
    number = 0
    number_snapshots = 0
    for time_window in code_window_map[ATA6code].keys():
        number += len(code_window_map[ATA6code][time_window])
        number_snapshots += sum(code_window_map[ATA6code][time_window])
    num_snapshots.append({'ATA6':ATA6code, 'NumSequences':number, 'NumSnapshots':number_snapshots})

pandas.DataFrame(num_snapshots)

,ATA6,NumSequences,NumSnapshots
0,0,3,1500
1,45008,1,500
2,44004,4,2000
3,45002,2,1000
4,43004,1,500
5,45021,9,4500
6,45007,1,500


In [ ]:
def get_snapshot_count(selected_codes, repairs, snapshots):
    
    s_veh_key = 'Veh Ref ID'
    r_veh_key = 'Chassis Reference Number'
    s_time_key = 'Event DateTime'
    r_time_key = 'Rpr_Dt'
    code = 'ATA6'
    
    snapshot_table = []
    total_snapshots = []
    
    # Must sort repair by their repair date so that after groupby('Chassis Reference Number')
    # repairs are accessed in order of increasing repair date.
    # This ensures start and end are set in the right order to calculate #snapshots correctly
    repairs = repairs.sort_values(by = r_time_key)
    
    start_date = pandas.to_datetime('1/1/2000')

    for ata6 in selected_codes:
        if ata6 == 0:
            continue        

        repair_group = repairs[(repairs[code] == ata6)].groupby(by=r_veh_key)
        for veh_id, row in repair_group:
            v_snapshots = snapshots[snapshots[s_veh_key] == veh_id].sort_values(by=s_time_key)
            start = start_date
            num_snapshots = 0
            num_repairs = 0
            for end in row[r_time_key]:
                range_mask = (v_snapshots[s_time_key] >= start) & (v_snapshots[s_time_key] <= end)
                num_snapshots = sum(range_mask)
                num_repairs += 1
                start = end
                snapshot_table.append({code:ata6, r_veh_key: veh_id, r_time_key:end, 'Num_Snapshots':num_snapshots})

    for ata6, row in pandas.DataFrame(snapshot_table).groupby(code):
        total_snapshots.append({code:ata6,'Total_Snapshots':row['Num_Snapshots'].sum()})
    
    snapshot_table = pandas.DataFrame(snapshot_table)
    total_snapshots = pandas.DataFrame(total_snapshots)
    
    return (snapshot_table, total_snapshots)

In [ ]:
# Getting snapshot counts
selected_codes = [0,45021, 44004, 43004, 45008, 45002, 45007] # first code is the ok
selected_codes = list(np.sort(selected_codes))

(ata6_train_snapshot_breakdown, ata6_train_snapshot_total) = get_snapshot_count(selected_codes, ata6_train_repairs, snapshots)
(ata6_val_snapshot_breakdown, ata6_val_snapshot_total) = get_snapshot_count(selected_codes, ata6_val_repairs, snapshots)
(ata6_test_snapshot_breakdown, ata6_test_snapshot_total) = get_snapshot_count(selected_codes, ata6_test_repairs, snapshots)

ata6_train_snapshot_breakdown.to_pickle('ata6_train_snapshot_breakdown.pkl')
ata6_val_snapshot_breakdown.to_pickle('ata6_val_snapshot_breakdown.pkl')
ata6_test_snapshot_breakdown.to_pickle('ata6_test_snapshot_breakdown.pkl')
ata6_train_snapshot_total.to_pickle('ata6_test_snapshot_total.pkl')
ata6_val_snapshot_total.to_pickle('ata6_val_snapshot_total.pkl')
ata6_test_snapshot_total.to_pickle('ata6_test_snapshot_total.pkl')

In [ ]:
s_veh_key = 'Veh Ref ID'
r_veh_key = 'Chassis Reference Number'
s_time_key = 'Event DateTime'
r_time_key = 'Rpr_Dt'
code = 'ATA6'

selected_codes = [0,45021, 44004, 43004, 45008, 45002, 45007] # first code is the ok
selected_codes = list(np.sort(selected_codes))

In [ ]:
Y_repair = np.zeros((len(snapshots),6))
Y_ok = np.zeros(len(ok_snapshots))


In [5]:
sum(snapshots['Veh Ref ID'] == 718)

774